## Environments

### Imports

In [5]:
import os
import pandas as pd #type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt
import sys

# additional paths
feature_path = os.path.join(os.getcwd(), '..', 'src', 'features')
visualize_path = os.path.join(os.getcwd(), '..', 'src', 'visualization')
sys.path.append(feature_path)
sys.path.append(visualize_path)


import build_features # type: ignore
import visualize # type: ignore
from sklearn.feature_selection import VarianceThreshold #type: ignore

In [6]:
path = os.path.join('..', 'data', 'interim', '01_H_Contortus_processed.pkl')
df = pd.read_pickle(path)
df.head()

,SMILES,ACTIVITY,ACTIVITY_SCORE,ACTIVATION_AT_6.8uM
PUBCHEM_RESULT_TAG,,,,
1,CC1=CC(=NC(=N1)NC2=NC(=CS2)C3=CC=CC=C3)C,1,100.0,36.85
2,CC1=C2C(=CC=C1)C(=C3C(=CC(=NC3=N2)C)C)N,1,84.0,31.28
3,CN1C2=CC=CC=C2N(C1=N)CC(=O)C3=CC=C(C=C3)Br.Br,1,77.0,28.61
4,C1=CC=C2C(=C1)C(=C3C=CC=CC3=N2)NCCCO,1,77.0,28.51
5,CC1=CC2=C(C=C1)N=C(N2)C3=C(C4=C(S3)N=C(C=C4)C)N,1,76.0,28.1


In [71]:
df = df[(df.ACTIVITY_SCORE > 20) | (df.ACTIVITY_SCORE < 14)]

In [72]:
df.ACTIVITY.value_counts()

ACTIVITY
1    1389
0    1188
Name: count, dtype: int64

## Loading Mordred descriptors dataset

In [73]:
path = os.path.join('..', 'data', 'interim', '02_H_Contortus_mordred_descriptors.pkl')

mordred_df = pd.read_pickle(path)
mordred_df.head()

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,ACTIVITY
PUBCHEM_RESULT_TAG,,,,,,,,,,,,,,,,,,,,,
1,0,0,25.802442,2.365976,4.651381,25.802442,1.290122,3.932435,4.097324,0.204866,...,66.954464,282.093917,8.296880,886,26,104.0,119.0,5.666667,4.361111,1
2,0,0,22.878917,2.509871,5.019743,22.878917,1.271051,3.847343,3.889241,0.216069,...,51.925327,237.126597,7.185654,532,33,100.0,122.0,6.388889,3.833333,1
3,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70.666210,422.958186,11.431302,2100000928,34,112.0,134.0,NaN,4.583333,1
4,0,0,25.434668,2.474546,4.949092,25.434668,1.338667,3.881497,3.768225,0.198328,...,52.493641,252.126263,7.203608,674,30,98.0,116.0,4.805556,4.361111,1
5,0,0,27.566121,2.512375,4.789613,27.566121,1.312672,4.018370,4.162911,0.198234,...,71.719636,294.093917,8.402683,904,34,120.0,146.0,6.250000,4.388889,1


Splitting dataset into train and test

In [74]:
from sklearn.model_selection import train_test_split 

# splitting into train and test data
X = mordred_df.loc[:, mordred_df.columns != 'ACTIVITY']
y = mordred_df['ACTIVITY']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=104, test_size=0.3, shuffle=True)


print('Total train samples: ', y_train.count())
print('Total test samples: ', y_test.count())

Total train samples:  2391
Total test samples:  1025


## Pre processing

Checking for outliers

In [75]:
X_train.dtypes.unique()

array([dtype('int64'), dtype('float64'), dtype('bool')], dtype=object)

In [76]:
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median').fit(X_train)
new_X_train = pd.DataFrame(imputer.transform(X_train))
new_X_train.columns = X_train.columns
new_X_train.index = X_train.index


clf = IsolationForest(random_state=0).fit(new_X_train)
new_X_train['outlier'] = clf.predict(new_X_train)
new_X_train = new_X_train[new_X_train['outlier'] == 1]
new_X_train.head()


absurd_index = new_X_train.Diameter[new_X_train.Diameter > 100000].index
new_X_train.drop(index=absurd_index, inplace=True)
inlier_index = new_X_train.index

In [77]:
new_X_train = X_train.loc[inlier_index, :]
new_X_train.shape

(2224, 1507)

In [78]:
# scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(new_X_train)
scaled_X_train = pd.DataFrame(scaler.transform(new_X_train))
scaled_X_train.columns = new_X_train.columns
scaled_X_train.index = new_X_train.index
scaled_X_train.head()

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
PUBCHEM_RESULT_TAG,,,,,,,,,,,,,,,,,,,,,
2292,-0.225046,-0.277931,1.817224,-0.501789,-0.437306,1.817224,-0.498292,1.727837,1.687331,-1.410373,...,0.675712,1.668076,2.336284,0.266402,2.331971,1.188662,1.563715,1.398655,2.134195,2.133010
1942,-0.225046,-0.277931,2.175693,1.507697,2.391140,2.175693,-0.003123,2.060381,0.370469,-2.175272,...,1.926667,1.594479,2.208196,-0.023416,2.451768,2.205719,2.215842,2.181704,2.115937,2.099847
1213,-0.225046,-0.277931,-0.097432,-0.908899,-0.396086,-0.097432,0.309546,-0.095726,-0.447620,-0.221674,...,-0.179545,-1.362847,-0.042604,-0.124993,-0.102856,-0.209791,-0.206345,-0.279308,-0.385438,-0.155251
1848,-0.225046,3.926964,0.404863,-0.221718,0.396493,0.404863,0.201301,0.417370,-0.323969,-0.819584,...,0.345083,-1.055399,-0.005299,-0.668375,0.217639,0.425870,0.259460,0.205436,0.344890,0.408523
649,-0.225046,-0.277931,-0.598503,0.270655,-0.808794,-0.598503,-1.408767,-0.485697,-0.781811,-0.048362,...,-0.578321,0.382664,-0.189243,0.111223,-0.468469,-0.464055,-0.578989,-0.540325,0.837862,-0.254741


In [79]:
# remove low variance features

mask = VarianceThreshold(threshold=0.1).fit(scaled_X_train).get_support()
low_var = scaled_X_train[scaled_X_train.columns[mask] ]
low_var.head()

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
PUBCHEM_RESULT_TAG,,,,,,,,,,,,,,,,,,,,,
2292,-0.225046,-0.277931,1.817224,-0.501789,-0.437306,1.817224,-0.498292,1.727837,1.687331,-1.410373,...,0.675712,1.668076,2.336284,0.266402,2.331971,1.188662,1.563715,1.398655,2.134195,2.133010
1942,-0.225046,-0.277931,2.175693,1.507697,2.391140,2.175693,-0.003123,2.060381,0.370469,-2.175272,...,1.926667,1.594479,2.208196,-0.023416,2.451768,2.205719,2.215842,2.181704,2.115937,2.099847
1213,-0.225046,-0.277931,-0.097432,-0.908899,-0.396086,-0.097432,0.309546,-0.095726,-0.447620,-0.221674,...,-0.179545,-1.362847,-0.042604,-0.124993,-0.102856,-0.209791,-0.206345,-0.279308,-0.385438,-0.155251
1848,-0.225046,3.926964,0.404863,-0.221718,0.396493,0.404863,0.201301,0.417370,-0.323969,-0.819584,...,0.345083,-1.055399,-0.005299,-0.668375,0.217639,0.425870,0.259460,0.205436,0.344890,0.408523
649,-0.225046,-0.277931,-0.598503,0.270655,-0.808794,-0.598503,-1.408767,-0.485697,-0.781811,-0.048362,...,-0.578321,0.382664,-0.189243,0.111223,-0.468469,-0.464055,-0.578989,-0.540325,0.837862,-0.254741


In [80]:
# removing percentage of missing continuous values
missing_percentage = 0.7

row_length = low_var.shape[0]
missing_length_threshold = round(row_length * missing_percentage, 0)
print("missing values greater than: ",missing_length_threshold)

resulting_columns = low_var.isna().sum()[low_var.isna().sum() < missing_length_threshold].index
low_var = low_var[resulting_columns]
low_var.shape

missing values greater than:  1557.0


(2224, 1296)

Removal of highly correlated features

In [81]:
def get_collinears(dataset, threshold):
    collinear_cols = set()
    corr = dataset.corr()
    for i in range(len(corr.columns)):
        for j in range(i):
            if abs(corr.iloc[i,j]) > threshold:
                colname = corr.columns[i]
                collinear_cols.add(colname)
    
    return list(collinear_cols)

collinear_columns = get_collinears(low_var, 0.9)    
print('number of collinear columns: ', len(collinear_columns))

number of collinear columns:  791


In [82]:
non_collinear_df = low_var.drop(collinear_columns, axis = 1)
non_collinear_df.shape

(2224, 505)

In [83]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer( ).fit(non_collinear_df)
imputed_df = pd.DataFrame(impute.transform(non_collinear_df))
imputed_df.columns = non_collinear_df.columns
imputed_df.index = non_collinear_df.index

RFECV

In [85]:
# from sklearn.feature_selection import RFE
# from sklearn.svm import SVC


# estimator = SVC(kernel="linear")
# selector = RFE(estimator)

# selector = selector.fit(imputed_df, y_train.loc[imputed_df.index])
# selector.support_


In [86]:
imputed_df

,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpMAD_A,VE1_A,VE2_A,VR1_A,nAromAtom,...,JGI3,JGI4,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,TopoShapeIndex,TSRW10
PUBCHEM_RESULT_TAG,,,,,,,,,,,,,,,,,,,,,
2292,-0.225046,-0.277931,1.817224,-0.501789,-0.437306,-0.498292,1.687331,-1.410373,0.029059,0.501009,...,0.719396,-0.422477,-0.262906,0.541272,-0.304799,0.123731,-0.058712,0.002838,0.951369,1.668076
1942,-0.225046,-0.277931,2.175693,1.507697,2.391140,-0.003123,0.370469,-2.175272,5.374721,-0.614908,...,0.538649,1.657716,-0.482708,1.652365,0.423223,0.382931,0.252021,0.098792,0.951369,1.594479
1213,-0.225046,-0.277931,-0.097432,-0.908899,-0.396086,0.309546,-0.447620,-0.221674,-0.138733,-0.614908,...,-1.408964,0.743394,-0.581837,-1.412854,-0.734876,0.675046,-0.164123,0.457882,-0.715343,-1.362847
1848,-0.225046,3.926964,0.404863,-0.221718,0.396493,0.201301,-0.323969,-0.819584,0.023290,0.501009,...,-0.144521,-0.237188,-0.847256,0.097212,0.240665,-0.305417,-0.585432,-0.089904,0.951369,-1.055399
649,-0.225046,-0.277931,-0.598503,0.270655,-0.808794,-1.408767,-0.781811,-0.048362,-0.213525,-1.172867,...,1.800776,0.676446,0.168827,1.220845,0.766686,0.512208,-0.431559,-0.256642,-0.993128,0.382664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215,-0.225046,-0.277931,-0.415259,-0.457217,-0.689711,-0.103261,-1.196942,-0.308212,-0.097077,0.222029,...,0.305779,-0.254879,-0.878793,-0.683447,-0.938461,1.654058,-0.830019,0.809032,0.951369,0.181432
1276,-0.225046,-0.277931,-0.791207,-0.199244,-0.390332,0.989347,-0.342046,1.126744,-0.307132,0.779988,...,-0.728110,-1.789767,0.819324,-0.285667,-1.983148,-1.219577,-0.023687,-1.652876,0.951369,-0.157429
730,-0.225046,-0.277931,-0.846005,0.210443,-0.395691,0.555568,-0.225075,1.208097,-0.313356,-0.893887,...,1.382537,-1.289797,-0.385384,-0.159538,0.115975,0.472207,0.233780,1.794616,0.951369,0.147367


In [87]:
keep_columns = imputed_df.columns

In [88]:
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

preprocessor = Pipeline([
    ('selector', ColumnTransformer([('selector', 'passthrough', keep_columns)], remainder='drop')),
    ('scaler', StandardScaler()),
    ('impute', SimpleImputer()),
]).fit(X_train.loc[inlier_index], y_train.loc[inlier_index])

In [92]:
svc_model = Pipeline([
    ('selector', ColumnTransformer([('selector', 'passthrough', keep_columns)], remainder='drop')),
    ('scaler', StandardScaler()),
    ('impute', SimpleImputer()),
    # ('imbalance', RandomUnderSampler()),
    ('svc', SVC(probability=True))
]).fit(X_train.loc[inlier_index], y_train.loc[inlier_index])

rfc_model = Pipeline([
    ('selector', ColumnTransformer([('selector', 'passthrough', keep_columns)], remainder='drop')),
    ('scaler', StandardScaler()),
    ('impute', SimpleImputer()),
    # ('imbalance', SMOTE()),
    ('rfc', RandomForestClassifier())
]).fit(X_train.loc[inlier_index], y_train.loc[inlier_index])

knn_model = Pipeline([
    ('selector', ColumnTransformer([('selector', 'passthrough', keep_columns)], remainder='drop')),
    ('scaler', StandardScaler()),
    ('impute', SimpleImputer()),
    # ('imbalance', SMOTE()),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]).fit(X_train.loc[inlier_index], y_train.loc[inlier_index])

lr_model = Pipeline([
    ('selector', ColumnTransformer([('selector', 'passthrough', keep_columns)], remainder='drop')),
    ('scaler', StandardScaler()),
    ('impute', SimpleImputer()),
    # ('imbalance', SMOTE()),
    ('lr', LogisticRegression())
]).fit(X_train.loc[inlier_index], y_train.loc[inlier_index])



d:\Users\Gabriel Anyaele\Desktop\daf12-antihelminthic-drugdiscovery\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Model Evaluation

In [41]:
from sklearn.metrics import roc_auc_score
import math

rfc_pred = rfc_model.predict(X_test)

print(round(roc_auc_score(y_test, rfc_model.predict_proba(X_test)[:, 1]), 3))
(visualize.score(y_test, rfc_pred))

0.775


{'accuracy_score': 0.713170731707317,
 'precision_score': 0.7314685314685314,
 'recall_score': 0.8368,
 'f1_score': 0.7805970149253731}

In [91]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test, rfc_pred).ravel()
(tn, fp, fn, tp)

(209, 191, 103, 522)

In [93]:
svc_pred = svc_model.predict(X_test)

print(round(roc_auc_score(y_test, svc_model.predict_proba(X_test)[:, 1]), 3))
(visualize.score(y_test, svc_pred))

0.774


{'accuracy_score': 0.713170731707317,
 'precision_score': 0.7215528781793842,
 'recall_score': 0.8624,
 'f1_score': 0.7857142857142857}

In [94]:
knn_pred = knn_model.predict(X_test)

print(round(roc_auc_score(y_test, knn_model.predict_proba(X_test)[:, 1]), 3))
(visualize.score(y_test, knn_pred))

0.689


{'accuracy_score': 0.6741463414634147,
 'precision_score': 0.6738351254480287,
 'recall_score': 0.9024,
 'f1_score': 0.771545827633379}

In [95]:
lr_pred = lr_model.predict(X_test)

print(round(roc_auc_score(y_test, lr_model.predict_proba(X_test)[:, 1]), 3))
(visualize.score(y_test, lr_pred))

0.694


{'accuracy_score': 0.6858536585365854,
 'precision_score': 0.7218155197657394,
 'recall_score': 0.7888,
 'f1_score': 0.753822629969419}

## saving models and datasets

In [96]:
# Saving our train and test dataset
import pickle
pickle_path = os.path.join('..', 'data', 'interim', 'train_data.pkl')
test_path = os.path.join('..', 'data', 'interim', 'eval_data.pkl')

final_train = X_train.loc[inlier_index]
final_train['Activity'] = y_train.loc[inlier_index]

final_test = X_test.copy()
final_test['Activity'] = y_test

pickle.dump(final_train, open(pickle_path, 'wb'))
pickle.dump(final_test, open(test_path, 'wb'))

In [97]:
# Saving models
base_model_path = os.path.join('..', 'models')

pickle.dump(rfc_model, open(os.path.join(base_model_path, 'rfc_model.pkl'), 'wb'))
pickle.dump(svc_model, open(os.path.join(base_model_path, 'svc_model.pkl'), 'wb'))
pickle.dump(knn_model, open(os.path.join(base_model_path, 'knn_model.pkl'), 'wb'))
pickle.dump(lr_model, open(os.path.join(base_model_path, 'lr_model.pkl'), 'wb'))